In [1]:
import pygame
import sys
import random
import numpy as np
import time
import copy
from numpy.core.multiarray import ndarray
from pygame.math import Vector2
from typing import List

def relu(a):
    return np.maximum(0, a)

def softmax(x):
    x = x - np.max(x, axis=1, keepdims=True)     # numerisk stabilisering
    exp_x = np.exp(x)
    return exp_x / np.sum(exp_x, axis=1, keepdims=True)

pygame 2.6.1 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists
/var/folders/bv/g95mxcxd5_z7kmmjlfm2_xgh0000gn/T/ipykernel_78217/1025312044.py:7: DeprecationWarning: numpy.core.multiarray is deprecated and has been renamed to numpy._core.multiarray. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.

# Vector i Snake

-------------------------
|Metode	|   Brug i Snake |
|--------------------------------|
|__init__	|   repræsenterer en position på banen |
|--------------------------------|
|__add__	 |  bevægelse → slangehoved + retning |
|--------------------------------|
|within	|   tjek vægge / game over |
|--------------------------------|
|__eq__	|   rammer slangen maden? |
|--------------------------------|
|random_within	|   placer maden tilfældigt |
|--------------------------------|

Vector-klassen er simpel, men den gør snake-koden:

* kortere

* mere læsbar

* mindre fejlbehæftet

* nemmere at udvide

In [5]:
class Vector:
    # Constructor
    def __init__(self, x: int = 0, y: int = 0):
        self.x = x
        self.y = y

    def __str__(self):
        # Text representation
        return f'Vector({self.x}, {self.y})'

    def __add__(self, other: 'Vector') -> 'Vector':
        # Addition af to vektorer - essensen i Snake - ny position = gammel position + bevægelse
        return Vector(self.x + other.x, self.y + other.y)

    def within(self, scope: 'Vector') -> bool:
        # Within(scope) tjekker om vektoren er inde i banen
        return self.x <= scope.x and self.x >= 0 and self.y <= scope.y and self.y >= 0

    def __eq__(self, other: 'Vector') -> bool:
        # Sammenligning af to vektorer - er to positioner ens?
        return self.x == other.x and self.y == other.y

    @classmethod
    def random_within(cls, scope: 'Vector') -> 'Vector':
        # lav en tilfældig posistion indenfor scope (banen)
        return Vector(random.randint(0, scope.x - 1), random.randint(0, scope.y - 1))

In [6]:
class SnakeGame:
    # Constructor når spillet startes
    def __init__(self, xsize: int = 30, ysize: int = 30, scale: int = 15):
        self.grid = Vector(xsize, ysize) # bane størrelse (antallet af ruder på grid'et)
        self.scale = scale # grid feltet bliver 15x15 pixels stort
        # Pygame opstart
        pygame.init()
        self.screen = pygame.display.set_mode((xsize * scale, ysize * scale)) # Opretter et vindue der er 450×450 pixels (30 × 15)
        self.clock = pygame.time.Clock() # Opretter en clock til at styre spillets hastighed (ticks per second)
        # farver
        self.color_snake_head = (0, 255, 0)
        self.color_food = (255, 0, 0)

    def __del__(self):
        # Når spillet afsluttes → luk pygame korrekt
        pygame.quit()

    def block(self, obj):
        # block() — konverterer et grid-punkt til en pixel-rectangle
        return (obj.x * self.scale, obj.y * self.scale, self.scale, self.scale)

    def run(self):
        # run() — selve spil-loopet
        running = True # begynder med et sandt flag “spillet kører”
        snake = Snake(game=self) # Opret en slange
        food = Food(game=self) # Opret en mad på en tilfældig position

        while running: # Game loop — kører 10 gange per sekund

            # handle pygame events (håndter input)
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    running = False
                if event.type == pygame.KEYDOWN: # ændre slange retning baseret på piletaster
                    if event.key == pygame.K_LEFT:
                        snake.v = Vector(-1, 0)
                    if event.key == pygame.K_RIGHT:
                        snake.v = Vector(1, 0)
                    if event.key == pygame.K_UP:
                        snake.v = Vector(0, -1)
                    if event.key == pygame.K_DOWN:
                        snake.v = Vector(0, 1)

            # wipe screen
            self.screen.fill('black')

            # update game state
            snake.move()
            if not snake.p.within(self.grid):
                running = False
            if snake.cross_own_tail:
                running = False
            if snake.p == food.p:
                snake.add_score()
                food = Food(game=self)

            # render game (grafics)
            for i, p in enumerate(snake.body):
                pygame.draw.rect(self.screen,
                                (0, max(128, 255 - i * 8), 0),
                                self.block(p))
            pygame.draw.rect(self.screen, self.color_food, self.block(food.p))

            # Update screen
            pygame.display.flip()

            # progress time (controle the speed of the game)
            self.clock.tick(10) # 60 = hurtig og 10 = langsom

        print(f'Score: {snake.score}')

In [7]:
class Food:
    # Constructor
    def __init__(self, game: SnakeGame): # Den modtager spillobjektet fra SnakeGame
        self.game = game # fortæller Food hvor stor banen er
        self.p = Vector.random_within(self.game.grid) # finder en tilfældig position indenfor banen

In [ ]:
class Snake:
    def __init__(self, *, game: SnakeGame):
        self.game = game # Slangen får adgang til hele spillet, fx grid-størrelse og settings
        self.score = 0 # Starter på 0, stiger når slangen spiser mad
        self.v = Vector(0, 0) # slangen starter uden bevægelse, når brugeren trykker - ændres v til henholdsvis (-1,0),(1,0),(0,-1),(0,1)
        self.body = deque() # deque er en dobbelt-ended kø — perfekt fordi slangen: får nyt hoved forrest (appendleft) smider halen bagerst (pop)
        self.body.append(Vector.random_within(self.game.grid)) # start position

    def move(self):
        # slage bevægelse
        # self.p henter slangens hoved
        # self.p + self.v lægger retningen til ny position
        self.p = self.p + self.v # Slangen bevæger sig ét grid-felt i retningen v

    # Kollision med egen hale
    @property
    def cross_own_tail(self):
        try:
            self.body.index(self.p, 1)
            return True
        except ValueError:
            return False

    # Accessors for p (hovedets position)
    # Getter
    @property
    def p(self): # Slangens hoved er første element i body-listen
        return self.body[0]

    # Setter
    @p.setter
    def p(self, value): # Slangen bevæger sig frem uden at ændre længde
        self.body.appendleft(value) # Ny hovedposition tilføjes forrest i body
        self.body.pop() # Sidste led fjernes (halen)

    # Slangen vokser når den spiser
    def add_score(self):
        self.score += 1
        tail = self.body.pop()
        self.body.append(tail)
        self.body.append(tail)

    def debug(self):
        # Debugging - viser slangens segmenter
        print('===')
        for i in self.body:
            print(str(i))